# Pulizia dei database Redis

Questo notebook guida attraverso la connessione a tutte le istanze Redis utilizzate dal progetto e ne cancella completamente il contenuto. È pensato per gli ambienti di sviluppo o di staging in cui è necessario ripartire da un database vuoto.

> ⚠️ **Attenzione:** l'operazione di pulizia è irreversibile. Assicurati di avere un backup aggiornato prima di procedere.


## Prerequisiti

* Un server Redis raggiungibile (di default in ascolto su `localhost:6379`).
* Le credenziali di accesso (se richieste dal server Redis).
* L'ambiente Python deve includere la libreria `redis` e il pacchetto del progetto presente in questo repository.


In [ ]:
import redis
from redis_helpers.utils import clean_all_databases

print("Librerie importate correttamente.")


In [ ]:
# Parametri di connessione (modifica secondo le tue necessità)
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = None  # Imposta la password se il server la richiede
USE_SSL = False        # Imposta a True se il server Redis è esposto tramite TLS/SSL

# Mappatura delle basi dati utilizzate dal progetto
DB_MAPPING = {
    "DATA": 0,
    "CAN": 1,
    "R": 2,
    "NR": 3,
    "CAR": 4,
    "AR": 5,
    "GP": 6,
    "BP": 7,
    "PR": 8,
    "AP": 9,
    "LOGS": 10,
}

BASE_REDIS_CONFIG = {
    "host": REDIS_HOST,
    "port": REDIS_PORT,
    "decode_responses": True,
    "ssl": USE_SSL,
}

if REDIS_PASSWORD not in (None, ""):
    BASE_REDIS_CONFIG["password"] = REDIS_PASSWORD

print("Configurazione di base pronta. Modifica i valori sopra se necessario.")


In [ ]:
def connect_to_databases(base_config, db_mapping):
    '''Crea una connessione Redis per ogni voce della mappatura.'''
    connections = {}
    for name, db_id in db_mapping.items():
        params = dict(base_config)
        params["db"] = db_id
        try:
            conn = redis.Redis(**params)
            conn.ping()
        except redis.RedisError as exc:
            raise RuntimeError(f"Connessione a Redis DB {db_id} ({name}) fallita: {exc}") from exc
        connections[name] = conn
        print(f"Connesso a Redis DB {db_id} ({name})")
    print(f"\nTotale connessioni attive: {len(connections)}")
    return connections


def describe_database_sizes(connections):
    '''Restituisce il numero di chiavi presenti in ciascun database.'''
    summary = {}
    for name, conn in connections.items():
        try:
            summary[name] = conn.dbsize()
        except redis.RedisError as exc:
            summary[name] = f"Errore: {exc}"
    return summary


connections = connect_to_databases(BASE_REDIS_CONFIG, DB_MAPPING)

pre_cleanup = describe_database_sizes(connections)
print("\nChiavi presenti prima della pulizia:")
for name, size in pre_cleanup.items():
    print(f" - {name} (DB {DB_MAPPING[name]}): {size}")


## Conferma manuale

Per evitare cancellazioni accidentali, esegui la cella seguente e digita `SI` quando richiesto. Qualsiasi altro input annullerà l'operazione.


In [ ]:
confirmation = input("Digita 'SI' per procedere con la pulizia completa dei database: ").strip().upper()
if confirmation != "SI":
    raise SystemExit("Operazione annullata dall'utente.")

clean_all_databases(connections, DB_MAPPING)


In [ ]:
post_cleanup = describe_database_sizes(connections)
print("\nChiavi presenti dopo la pulizia:")
for name, size in post_cleanup.items():
    print(f" - {name} (DB {DB_MAPPING[name]}): {size}")


Se tutte le basi dati riportano `0` chiavi, la pulizia è stata completata con successo.
